# Démonstration : Firewalls et Filtrage de Paquets

Ce notebook simule le fonctionnement d'un firewall et présente des règles de filtrage.

## 📚 Concepts couverts

1. **Firewall à filtrage de paquets** (stateless)
2. **Firewall stateful** (suivi de connexions)
3. **Règles iptables** (syntaxe et configuration)
4. **Web Application Firewall (WAF)** basique
5. **Discussion critique** des limitations

---

In [ ]:
import re
from typing import List, Dict, Tuple
from dataclasses import dataclass
from enum import Enum

## 1. Firewall à Filtrage de Paquets (Stateless)

Un firewall **stateless** examine chaque paquet indépendamment selon des règles.

### Critères de filtrage
- Adresse IP source/destination
- Port source/destination
- Protocole (TCP, UDP, ICMP)
- Flags TCP (SYN, ACK, FIN, etc.)

In [ ]:
class Action(Enum):
    ACCEPT = "ACCEPT"
    DROP = "DROP"
    REJECT = "REJECT"

@dataclass
class Packet:
    """Représente un paquet réseau simplifié"""
    src_ip: str
    dst_ip: str
    src_port: int
    dst_port: int
    protocol: str  # TCP, UDP, ICMP
    flags: List[str] = None  # SYN, ACK, FIN, RST, etc.
    
    def __post_init__(self):
        if self.flags is None:
            self.flags = []

@dataclass
class FirewallRule:
    """Règle de firewall"""
    chain: str  # INPUT, OUTPUT, FORWARD
    protocol: str = "*"  # *, TCP, UDP, ICMP
    src_ip: str = "*"    # IP ou "*" pour tous
    dst_ip: str = "*"
    src_port: int = None  # None pour "any"
    dst_port: int = None
    flags: List[str] = None
    action: Action = Action.ACCEPT
    description: str = ""
    
    def matches(self, packet: Packet, chain: str) -> bool:
        """Vérifie si le paquet correspond à la règle"""
        if self.chain != chain:
            return False
        
        # Vérifie protocole
        if self.protocol != "*" and self.protocol != packet.protocol:
            return False
        
        # Vérifie IP source
        if self.src_ip != "*" and not self._ip_matches(self.src_ip, packet.src_ip):
            return False
        
        # Vérifie IP destination
        if self.dst_ip != "*" and not self._ip_matches(self.dst_ip, packet.dst_ip):
            return False
        
        # Vérifie ports
        if self.src_port is not None and self.src_port != packet.src_port:
            return False
        
        if self.dst_port is not None and self.dst_port != packet.dst_port:
            return False
        
        # Vérifie flags TCP
        if self.flags:
            if not all(flag in packet.flags for flag in self.flags):
                return False
        
        return True
    
    @staticmethod
    def _ip_matches(rule_ip: str, packet_ip: str) -> bool:
        """Vérifie si l'IP correspond (supporte CIDR simple)"""
        if "/" in rule_ip:
            # Support basique CIDR (simplifié)
            network = rule_ip.split("/")[0]
            return packet_ip.startswith(".".join(network.split(".")[:3]))
        return rule_ip == packet_ip

In [ ]:
class StatelessFirewall:
    """Firewall stateless (examine chaque paquet indépendamment)"""
    
    def __init__(self, default_policy: Action = Action.DROP):
        self.rules = []
        self.default_policy = default_policy
        self.stats = {
            'accepted': 0,
            'dropped': 0,
            'rejected': 0
        }
    
    def add_rule(self, rule: FirewallRule):
        """Ajoute une règle"""
        self.rules.append(rule)
    
    def process_packet(self, packet: Packet, chain: str = "INPUT") -> Action:
        """Traite un paquet selon les règles"""
        # Parcourt les règles dans l'ordre
        for rule in self.rules:
            if rule.matches(packet, chain):
                # Première règle qui matche
                self._update_stats(rule.action)
                return rule.action
        
        # Aucune règle ne matche : applique la politique par défaut
        self._update_stats(self.default_policy)
        return self.default_policy
    
    def _update_stats(self, action: Action):
        if action == Action.ACCEPT:
            self.stats['accepted'] += 1
        elif action == Action.DROP:
            self.stats['dropped'] += 1
        elif action == Action.REJECT:
            self.stats['rejected'] += 1
    
    def print_rules(self):
        """Affiche les règles configurées"""
        print(f"\n📋 Règles du firewall (politique par défaut: {self.default_policy.value})")
        print("=" * 80)
        for i, rule in enumerate(self.rules, 1):
            print(f"{i:2d}. {rule.action.value:8s} | {rule.protocol:5s} | "
                  f"{rule.src_ip:15s}:{rule.src_port or '*':5} → "
                  f"{rule.dst_ip:15s}:{rule.dst_port or '*':5}")
            if rule.description:
                print(f"    {rule.description}")
    
    def print_stats(self):
        """Affiche les statistiques"""
        total = sum(self.stats.values())
        print(f"\n📊 Statistiques :")
        print(f"   Total paquets : {total}")
        print(f"   ✅ Acceptés   : {self.stats['accepted']} ({self.stats['accepted']/total*100:.1f}%)")
        print(f"   ❌ Droppés    : {self.stats['dropped']} ({self.stats['dropped']/total*100:.1f}%)")
        print(f"   🚫 Rejetés    : {self.stats['rejected']} ({self.stats['rejected']/total*100:.1f}%)")

In [ ]:
# Configuration d'un firewall basique
print("=" * 80)
print("DÉMONSTRATION: Firewall Stateless")
print("=" * 80)

firewall = StatelessFirewall(default_policy=Action.DROP)

# Règle 1: Autoriser SSH (port 22) depuis un réseau spécifique
firewall.add_rule(FirewallRule(
    chain="INPUT",
    protocol="TCP",
    src_ip="192.168.1.0/24",
    dst_port=22,
    action=Action.ACCEPT,
    description="Autoriser SSH depuis réseau local"
))

# Règle 2: Autoriser HTTP/HTTPS
firewall.add_rule(FirewallRule(
    chain="INPUT",
    protocol="TCP",
    dst_port=80,
    action=Action.ACCEPT,
    description="Autoriser HTTP"
))

firewall.add_rule(FirewallRule(
    chain="INPUT",
    protocol="TCP",
    dst_port=443,
    action=Action.ACCEPT,
    description="Autoriser HTTPS"
))

# Règle 3: Bloquer connexions depuis IP suspecte
firewall.add_rule(FirewallRule(
    chain="INPUT",
    src_ip="10.0.0.100",
    action=Action.DROP,
    description="Bloquer IP suspecte"
))

# Règle 4: Rejeter tout le reste (explicite)
# Non nécessaire car politique par défaut = DROP

firewall.print_rules()

# Test de paquets
print("\n" + "=" * 80)
print("TEST DE PAQUETS")
print("=" * 80)

test_packets = [
    (Packet("192.168.1.50", "192.168.1.10", 54321, 22, "TCP"), "SSH depuis réseau local"),
    (Packet("203.0.113.1", "192.168.1.10", 54322, 22, "TCP"), "SSH depuis Internet"),
    (Packet("203.0.113.1", "192.168.1.10", 54323, 80, "TCP"), "HTTP depuis Internet"),
    (Packet("203.0.113.1", "192.168.1.10", 54324, 443, "TCP"), "HTTPS depuis Internet"),
    (Packet("10.0.0.100", "192.168.1.10", 54325, 80, "TCP"), "Requête depuis IP bloquée"),
    (Packet("203.0.113.1", "192.168.1.10", 54326, 3306, "TCP"), "MySQL depuis Internet"),
]

for packet, description in test_packets:
    action = firewall.process_packet(packet)
    icon = "✅" if action == Action.ACCEPT else "❌"
    print(f"{icon} {action.value:8s} | {description}")
    print(f"   {packet.src_ip}:{packet.src_port} → {packet.dst_ip}:{packet.dst_port} ({packet.protocol})")

firewall.print_stats()

## 2. Firewall Stateful (Connection Tracking)

Un firewall **stateful** suit l'état des connexions TCP/UDP.

### États de connexion TCP
- **NEW** : Nouveau paquet SYN (début de connexion)
- **ESTABLISHED** : Connexion établie (échange bidirectionnel)
- **RELATED** : Paquet lié à une connexion existante (ex: FTP data)
- **INVALID** : Paquet malformé ou non attendu

In [ ]:
class ConnectionState(Enum):
    NEW = "NEW"
    ESTABLISHED = "ESTABLISHED"
    RELATED = "RELATED"
    INVALID = "INVALID"

class StatefulFirewall:
    """Firewall stateful avec suivi de connexions"""
    
    def __init__(self, default_policy: Action = Action.DROP):
        self.rules = []
        self.default_policy = default_policy
        # Table de connexions : {(src_ip, src_port, dst_ip, dst_port): state}
        self.connections = {}
        self.stats = {
            'accepted': 0,
            'dropped': 0,
            'new': 0,
            'established': 0
        }
    
    def add_rule(self, rule: FirewallRule, state: ConnectionState = None):
        """Ajoute une règle avec état de connexion optionnel"""
        self.rules.append((rule, state))
    
    def process_packet(self, packet: Packet, chain: str = "INPUT") -> Action:
        """Traite un paquet avec suivi de connexion"""
        conn_key = (packet.src_ip, packet.src_port, packet.dst_ip, packet.dst_port)
        conn_key_reverse = (packet.dst_ip, packet.dst_port, packet.src_ip, packet.src_port)
        
        # Détermine l'état de la connexion
        if conn_key in self.connections:
            conn_state = ConnectionState.ESTABLISHED
            self.stats['established'] += 1
        elif conn_key_reverse in self.connections:
            # Paquet de retour d'une connexion existante
            conn_state = ConnectionState.ESTABLISHED
            self.connections[conn_key] = True  # Bidirectionnel
            self.stats['established'] += 1
        elif "SYN" in packet.flags and "ACK" not in packet.flags:
            # Nouveau SYN = nouvelle connexion
            conn_state = ConnectionState.NEW
            self.stats['new'] += 1
        else:
            conn_state = ConnectionState.INVALID
        
        # Parcourt les règles
        for rule, required_state in self.rules:
            # Vérifie l'état de connexion si spécifié
            if required_state and conn_state != required_state:
                continue
            
            if rule.matches(packet, chain):
                if rule.action == Action.ACCEPT and conn_state == ConnectionState.NEW:
                    # Enregistre la nouvelle connexion
                    self.connections[conn_key] = True
                
                self._update_stats(rule.action)
                return rule.action
        
        # Politique par défaut
        self._update_stats(self.default_policy)
        return self.default_policy
    
    def _update_stats(self, action: Action):
        if action == Action.ACCEPT:
            self.stats['accepted'] += 1
        elif action == Action.DROP:
            self.stats['dropped'] += 1
    
    def print_connections(self):
        """Affiche les connexions trackées"""
        print(f"\n🔗 Connexions trackées : {len(self.connections)}")
        for i, conn in enumerate(list(self.connections.keys())[:10], 1):
            src_ip, src_port, dst_ip, dst_port = conn
            print(f"   {i}. {src_ip}:{src_port} → {dst_ip}:{dst_port}")

In [ ]:
# Configuration firewall stateful
print("=" * 80)
print("DÉMONSTRATION: Firewall Stateful")
print("=" * 80)

stateful_fw = StatefulFirewall(default_policy=Action.DROP)

# Règle 1: Accepter nouvelles connexions HTTP/HTTPS
stateful_fw.add_rule(
    FirewallRule("INPUT", "TCP", dst_port=80, action=Action.ACCEPT),
    state=ConnectionState.NEW
)

stateful_fw.add_rule(
    FirewallRule("INPUT", "TCP", dst_port=443, action=Action.ACCEPT),
    state=ConnectionState.NEW
)

# Règle 2: Accepter tout le trafic ESTABLISHED (retours de connexions)
stateful_fw.add_rule(
    FirewallRule("INPUT", action=Action.ACCEPT),
    state=ConnectionState.ESTABLISHED
)

print("\n📋 Configuration :")
print("   1. Accepter nouvelles connexions HTTP/HTTPS (NEW)")
print("   2. Accepter tout le trafic ESTABLISHED")
print("   3. Politique par défaut: DROP")

# Simulation d'une connexion HTTP complète
print("\n" + "=" * 80)
print("SIMULATION: Connexion HTTP complète")
print("=" * 80)

# 1. Client envoie SYN (NEW)
syn_packet = Packet("203.0.113.1", "192.168.1.10", 54321, 80, "TCP", flags=["SYN"])
action = stateful_fw.process_packet(syn_packet)
print(f"\n1️⃣  Client → Server : SYN")
print(f"   État: NEW, Action: {action.value}")

# 2. Server répond SYN-ACK (ESTABLISHED)
synack_packet = Packet("192.168.1.10", "203.0.113.1", 80, 54321, "TCP", flags=["SYN", "ACK"])
action = stateful_fw.process_packet(synack_packet, chain="OUTPUT")
print(f"\n2️⃣  Server → Client : SYN-ACK")
print(f"   État: ESTABLISHED (retour), Action: {action.value}")

# 3. Client envoie ACK (ESTABLISHED)
ack_packet = Packet("203.0.113.1", "192.168.1.10", 54321, 80, "TCP", flags=["ACK"])
action = stateful_fw.process_packet(ack_packet)
print(f"\n3️⃣  Client → Server : ACK")
print(f"   État: ESTABLISHED, Action: {action.value}")

# 4. Échange de données (ESTABLISHED)
data_packet = Packet("203.0.113.1", "192.168.1.10", 54321, 80, "TCP", flags=["ACK"])
action = stateful_fw.process_packet(data_packet)
print(f"\n4️⃣  Client → Server : GET /index.html HTTP/1.1")
print(f"   État: ESTABLISHED, Action: {action.value}")

response_packet = Packet("192.168.1.10", "203.0.113.1", 80, 54321, "TCP", flags=["ACK"])
action = stateful_fw.process_packet(response_packet, chain="OUTPUT")
print(f"\n5️⃣  Server → Client : HTTP/1.1 200 OK")
print(f"   État: ESTABLISHED, Action: {action.value}")

stateful_fw.print_connections()

# Test d'un paquet non sollicité (devrait être bloqué)
print("\n" + "=" * 80)
print("TEST: Paquet non sollicité (sans SYN, pas de connexion existante)")
print("=" * 80)

unsolicited = Packet("203.0.113.50", "192.168.1.10", 12345, 22, "TCP", flags=["ACK"])
action = stateful_fw.process_packet(unsolicited)
print(f"\nPaquet ACK sans connexion établie")
print(f"État: INVALID, Action: {action.value}")
print("\n✅ Le firewall stateful bloque les paquets non sollicités!")

print(f"\n📊 Statistiques :")
print(f"   Nouvelles connexions : {stateful_fw.stats['new']}")
print(f"   Paquets ESTABLISHED  : {stateful_fw.stats['established']}")
print(f"   Acceptés             : {stateful_fw.stats['accepted']}")
print(f"   Droppés              : {stateful_fw.stats['dropped']}")

## 3. Règles iptables

Voici des exemples de règles **iptables** (firewall Linux standard).

In [ ]:
# Exemples de règles iptables (documentation)
iptables_examples = """
=============================================================================
EXEMPLES DE RÈGLES IPTABLES
=============================================================================

# 1. POLITIQUE PAR DÉFAUT (tout bloquer)
iptables -P INPUT DROP
iptables -P FORWARD DROP
iptables -P OUTPUT ACCEPT

# 2. AUTORISER LOOPBACK (127.0.0.1)
iptables -A INPUT -i lo -j ACCEPT
iptables -A OUTPUT -o lo -j ACCEPT

# 3. AUTORISER CONNEXIONS ÉTABLIES
iptables -A INPUT -m state --state ESTABLISHED,RELATED -j ACCEPT
iptables -A OUTPUT -m state --state ESTABLISHED,RELATED -j ACCEPT

# 4. AUTORISER SSH (port 22) depuis réseau local uniquement
iptables -A INPUT -p tcp -s 192.168.1.0/24 --dport 22 -j ACCEPT

# 5. AUTORISER HTTP/HTTPS
iptables -A INPUT -p tcp --dport 80 -j ACCEPT
iptables -A INPUT -p tcp --dport 443 -j ACCEPT

# 6. LIMITER CONNEXIONS SSH (protection brute-force)
iptables -A INPUT -p tcp --dport 22 -m state --state NEW -m recent --set
iptables -A INPUT -p tcp --dport 22 -m state --state NEW -m recent --update \\
         --seconds 60 --hitcount 4 -j DROP

# 7. BLOQUER UNE IP SPÉCIFIQUE
iptables -A INPUT -s 10.0.0.100 -j DROP

# 8. PROTECTION SYN FLOOD
iptables -N syn_flood
iptables -A INPUT -p tcp --syn -j syn_flood
iptables -A syn_flood -m limit --limit 1/s --limit-burst 3 -j RETURN
iptables -A syn_flood -j DROP

# 9. BLOQUER PING (ICMP Echo Request)
iptables -A INPUT -p icmp --icmp-type echo-request -j DROP

# 10. LOGGER LES PAQUETS DROPPÉS
iptables -A INPUT -m limit --limit 5/min -j LOG --log-prefix "iptables dropped: " \\
         --log-level 7
iptables -A INPUT -j DROP

# 11. NAT (MASQUERADE pour routeur)
iptables -t nat -A POSTROUTING -o eth0 -j MASQUERADE

# 12. PORT FORWARDING (redirection)
iptables -t nat -A PREROUTING -p tcp --dport 8080 -j REDIRECT --to-port 80

=============================================================================
COMMANDES UTILES
=============================================================================

# Lister les règles
iptables -L -v -n

# Sauvegarder les règles
iptables-save > /etc/iptables/rules.v4

# Restaurer les règles
iptables-restore < /etc/iptables/rules.v4

# Supprimer toutes les règles (ATTENTION!)
iptables -F
iptables -X
iptables -t nat -F
iptables -t nat -X

# Compter les paquets
iptables -L -v -n --line-numbers

=============================================================================
"""

print(iptables_examples)

## 4. Web Application Firewall (WAF) Basique

Un **WAF** inspecte le contenu HTTP pour détecter des attaques applicatives (SQL injection, XSS, etc.).

In [ ]:
class WAF:
    """Web Application Firewall simplifié"""
    
    # Patterns d'attaque
    SQL_INJECTION_PATTERNS = [
        r"(\b(SELECT|UNION|INSERT|UPDATE|DELETE|DROP|CREATE|ALTER)\b.*\b(FROM|INTO|TABLE|WHERE)\b)",
        r"(--|#|/\*|\*/)",
        r"('\s*OR\s*'.*'\s*=\s*')",
        r"(;\s*DROP\s+TABLE)",
    ]
    
    XSS_PATTERNS = [
        r"(<script[^>]*>.*</script>)",
        r"(javascript:)",
        r"(onerror\s*=)",
        r"(onload\s*=)",
        r"(<iframe[^>]*>)",
    ]
    
    PATH_TRAVERSAL_PATTERNS = [
        r"(\.\.[\\/])",
        r"(%2e%2e[\\/])",
    ]
    
    def __init__(self):
        self.stats = {
            'requests': 0,
            'blocked': 0,
            'sql_injection': 0,
            'xss': 0,
            'path_traversal': 0
        }
    
    def inspect_request(self, method: str, path: str, query: str = "", body: str = "") -> Tuple[bool, str]:
        """Inspecte une requête HTTP"""
        self.stats['requests'] += 1
        
        full_request = f"{path} {query} {body}"
        
        # Vérifie SQL Injection
        for pattern in self.SQL_INJECTION_PATTERNS:
            if re.search(pattern, full_request, re.IGNORECASE):
                self.stats['blocked'] += 1
                self.stats['sql_injection'] += 1
                return False, "SQL Injection detected"
        
        # Vérifie XSS
        for pattern in self.XSS_PATTERNS:
            if re.search(pattern, full_request, re.IGNORECASE):
                self.stats['blocked'] += 1
                self.stats['xss'] += 1
                return False, "XSS detected"
        
        # Vérifie Path Traversal
        for pattern in self.PATH_TRAVERSAL_PATTERNS:
            if re.search(pattern, path, re.IGNORECASE):
                self.stats['blocked'] += 1
                self.stats['path_traversal'] += 1
                return False, "Path Traversal detected"
        
        return True, "OK"
    
    def print_stats(self):
        print(f"\n📊 Statistiques WAF :")
        print(f"   Total requêtes      : {self.stats['requests']}")
        print(f"   ❌ Bloquées         : {self.stats['blocked']} ({self.stats['blocked']/self.stats['requests']*100:.1f}%)")
        print(f"      SQL Injection    : {self.stats['sql_injection']}")
        print(f"      XSS              : {self.stats['xss']}")
        print(f"      Path Traversal   : {self.stats['path_traversal']}")

# Test du WAF
print("=" * 80)
print("DÉMONSTRATION: Web Application Firewall (WAF)")
print("=" * 80)

waf = WAF()

test_requests = [
    ("GET", "/index.html", "", "", "Requête légitime"),
    ("GET", "/users", "id=1", "", "Requête légitime avec paramètre"),
    ("GET", "/users", "id=1' OR '1'='1", "", "SQL Injection (OR 1=1)"),
    ("GET", "/search", "q=<script>alert('XSS')</script>", "", "XSS (script tag)"),
    ("GET", "/../../../etc/passwd", "", "", "Path Traversal"),
    ("POST", "/login", "", "username=admin&password='; DROP TABLE users;--", "SQL Injection (DROP TABLE)"),
    ("GET", "/page", "name=<img src=x onerror=alert(1)>", "", "XSS (onerror)"),
]

print("\n🧪 Tests :")
for method, path, query, body, description in test_requests:
    allowed, reason = waf.inspect_request(method, path, query, body)
    icon = "✅" if allowed else "❌"
    request_str = f"{method} {path}"
    if query:
        request_str += f"?{query[:50]}"
    print(f"\n{icon} {description}")
    print(f"   {request_str}")
    if not allowed:
        print(f"   🚫 Blocked: {reason}")

waf.print_stats()

## 5. Discussion Critique des Firewalls

### ✅ Avantages

1. **Première ligne de défense** contre les attaques réseau
2. **Réduction de la surface d'attaque** (ferme les ports inutilisés)
3. **Logging et audit** du trafic réseau
4. **Protection automatique** sans modification des applications

### ❌ Limitations

#### 1. **Trafic chiffré (HTTPS)**
- Les WAF ne peuvent pas inspecter le contenu HTTPS sans terminaison SSL
- Risque de man-in-the-middle si le WAF déchiffre le trafic

#### 2. **Attaques applicatives**
- Un firewall réseau ne protège pas contre :
  - Logique métier vulnérable
  - Authentication bypass
  - IDOR (Insecure Direct Object Reference)
  - Race conditions

#### 3. **Faux positifs**
- Les règles trop strictes bloquent du trafic légitime
- Exemple : `O'Reilly` détecté comme SQL injection à cause de `'`

#### 4. **Contournement**
- **Fragmentation** : Découper les payloads malveillants
- **Encoding** : Unicode, URL encoding, base64
- **Tunneling** : Encapsuler dans protocoles autorisés (DNS tunneling)

#### 5. **Performance**
- L'inspection approfondie (DPI) impacte les performances
- Trade-off entre sécurité et latence

#### 6. **Configuration complexe**
- Erreurs de configuration fréquentes
- Exemple : Oublier de bloquer IPv6 alors qu'IPv4 est filtré

### 🎯 Best Practices

1. **Defense in depth** : Le firewall est UNE couche, pas la seule
2. **Principe du moindre privilège** : Bloquer tout sauf ce qui est nécessaire
3. **Logging et monitoring** : Analyser les règles qui matchent
4. **Tests réguliers** : Auditer les règles et tester les contournements
5. **Mise à jour** : Patterns de WAF doivent être maintenus (nouvelles attaques)

---

## 🔗 Références

- **iptables** : https://www.netfilter.org/
- **nftables** : Successeur moderne d'iptables
- **OWASP ModSecurity** : WAF open-source
- **pfSense** : Firewall open-source basé sur FreeBSD
- **Snort** : IDS/IPS open-source avec règles de détection